# Predict difficulty of news articles

In [13]:
import pymongo
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Connect to MONGO DB

In [14]:
myclient = pymongo.MongoClient()
mydb = myclient["finalProject"]
main = mydb["main_processed_article"]

# Load the logistic regression model and the tfidf vectorizer

In [15]:
pickle_in = open("logistic_regression.pickle","rb")
model = pickle.load(pickle_in)

In [16]:
pickle_in = open("tfidf_vectorizer.pickle","rb")
tfidf = pickle.load(pickle_in)

# Iterate through the cursor, adding the predictions for each article

In [17]:
cursor = main.find({})



In [18]:
for article in tqdm(cursor):
    
    # First check if the article has already been classified
    myquery = { "_id": article['_id'] }
    
    already_classified = list(main.find(myquery, {"_id": 0, "level_binary": 1}))
    article_class = already_classified[0]['level_binary']
    
    # If it has not been classified yet then make a prediction and upate the corresponding field
    if not article_class:
    
        # Get the bag of words and make a prediction
        bag_of_words = article['bag_of_words']
        vectors = tfidf.transform(bag_of_words)
        article_prediction = model.predict(vectors)
        article_prediction = int(article_prediction[0])
        
        # See: https://www.w3schools.com/python/python_mongodb_update.asp
        newvalues = { "$set": { "level_binary": article_prediction } }
        main.update_one(myquery, newvalues)
    else:
        print("Article already classified")

65it [00:00, 646.19it/s]

Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
A

160it [00:00, 465.48it/s]

Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
A

364it [00:00, 641.24it/s]

...

Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
A

7415it [00:10, 1018.23it/s]

Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
A

7606it [00:10, 726.55it/s] 

Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
Article already classified
